In [8]:
from fenics import * 
import matplotlib.pyplot as plt
import numpy as np
import time

#suppress some output
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)

In [9]:
## Simulation and mesh parameters

# mesh size
Lx = 1.0         
Ly = 1.0

# set grid size
mx = 25
my = int(round(mx*Ly))

# number of points along k- slice
J = 3

# number of eigenvalues
#evals = 2*2*int(Lx*Ly)
evals = 3
E = np.zeros((J,evals))

In [10]:
## Mesh

# set up honeycomb lattice
q = 4*np.pi/(sqrt(3))
k1 = q*np.array([1,sqrt(3)])/2.0
k2 = q*np.array([1,-sqrt(3)])/2.0
k3 = -(k1+k2)
K = (k1-k2)/3

v1_h = np.array([sqrt(3),1])/2.0
v2_h = np.array([sqrt(3),-1])/2.0

# different edges
# a1 = 1; b1 = 1; a2 = 0; b2 = 1;     # armchair
a1, b1, a2, b2 = 1, 0, 0, 1       # zigzag
# a1 = 3; b1 = 2; a2 = 1; b2 = 1;     # general
# a1 = 2; b1 = 1; a2 = 1; b2 = 1;     # general

v1 = a1*v1_h + b1*v2_h
v2 = a2*v1_h + b2*v2_h

ktilde = -b1*k1+a1*k2         # general

# quasimomentum slice
quasimomentum = np.linspace(-0.2,0.0,J)
boundary_0 = K[0]*np.ones((1,J)) + ktilde[0]*quasimomentum
boundary_1 = K[1]*np.ones((1,J)) + ktilde[1]*quasimomentum
boundary = np.concatenate((boundary_0,boundary_1),axis=0)

# create mesh      
def honeycomb_cylinder(x, y):
    return [v1[0]*x + v2[0]*y, v1[1]*x + v2[1]*y]

mesh = RectangleMesh(Point(-Lx/2,-Ly/2),Point(Lx/2,Ly/2),mx,my, 'left') # “left”, “right”, “left/right”, “crossed”
x = mesh.coordinates()[:,0]
y = mesh.coordinates()[:,1]

x_hat, y_hat = honeycomb_cylinder(x, y)
xy_hat_coor = np.array([x_hat, y_hat]).transpose()
mesh.coordinates()[:] = xy_hat_coor
#plot(mesh), interactive()

In [11]:
## Boundary Conditions

# sub domain for Dirichlet boundary condition
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv
        return bool( ( near(y_r, -Ly/2.0) or near(y_r, Ly/2.0) ) and on_boundary)
    
# sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        return bool( ( near(x_r, -Lx/2.0) and near(x_r, -Lx/2.0) ) and on_boundary)

    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        x_h = x_r - (Lx - 0)
        y_h = y_r
        
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h
        
## perfectly periodic problem

# sub domain for Periodic boundary condition
class PerfectlyPeriodicBoundary(SubDomain):

    # left boundary is "target domain" G
    def inside(self, x, on_boundary):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        return bool( (near(x_r, -Lx/2.0) or near(y_r, -Ly/2.0)) and 
                    (not ((near(x_r, -Lx/2.0) and near(y_r, +Ly/2.0)) or 
                            (near(x_r, +Lx/2.0) and near(y_r, -Ly/2.0))) ) and on_boundary)
        
    # map right boundary (H) to left boundary (G)
    def map(self, x, y):
        detv = v1[0]*v2[1] - v2[0]*v1[1]
        x_r = (v2[1]*x[0] - v2[0]*x[1])/detv
        y_r = (-v1[1]*x[0] + v1[0]*x[1])/detv 
        
        if near(x_r, +Lx/2.0) and near(y_r, +Ly/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r - Ly - 0.0
        elif near(x_r, +Lx/2.0):
            x_h = x_r  - Lx - 0.0
            y_h = y_r
        else:   # near(y_r, +Ly/2.0)
            x_h = x_r 
            y_h = y_r - Ly - 0.0
            
        y[0] = v1[0]*x_h + v2[0]*y_h
        y[1] = v1[1]*x_h + v2[1]*y_h

In [12]:
## Finite Elements

# periodic = PeriodicBoundary()
periodic = PerfectlyPeriodicBoundary()

#V = FunctionSpace(mesh, "CG", 2, constrained_domain=periodic)

P2 = FiniteElement("P", triangle, 2)
element = MixedElement([P2, P2])
W = FunctionSpace(mesh, element, constrained_domain=periodic)

# define functions
(u_r, u_i) = TrialFunctions(W)
(v_r, v_i) = TestFunctions(W)

In [13]:
## Potential terms

# helper function to convert numpy array to ufl matrix type
def np_to_ufl_matrix(A):
    return as_matrix( [ [A[0][0], A[0][1]], [A[1][0], A[1][1]] ] )

# scalar case
#V_e = Expression('2.5-0.5*(cos(k10*x[0]+k11*x[1]) + cos(k20*x[0]+k21*x[1]) + cos(k30*x[0]+k31*x[1]))', \
#                k10=k1[0], k11=k1[1], k20=k2[0], k21=k2[1], k30=k3[0], k31=k3[1], degree=1)
#C = as_matrix( ((V_e, 0.0), (0.0, V_e)) )

# matrix cases
cosk1 = Expression('0.5*cos(k10*x[0]+k11*x[1])', k10=k1[0], k11=k1[1], degree=1)
cosk2 = Expression('0.5*cos(k20*x[0]+k21*x[1])', k20=k2[0], k21=k2[1], degree=1)
cosk3 = Expression('0.5*cos(k30*x[0]+k31*x[1])', k30=k3[0], k31=k3[1], degree=1)

c_np = [ [-1.0, 2.0], [-2.0, -1.0] ]
# c_np = [ [0.0, 0.0], [0.0, 0.0] ]
# c_np = [ [1.0, 0.0], [0.0, 1.0] ]
r_np = [ [-0.5, sqrt(3.0)/2.0], [-sqrt(3.0)/2.0, -0.5] ]
r_np_t = np.transpose(r_np)
rcrt_np = np.dot(np.dot(r_np,c_np), r_np_t)
rtcr_np = np.dot(np.dot(r_np_t,c_np), r_np)

c = np_to_ufl_matrix(c_np)
rcrt = np_to_ufl_matrix(rcrt_np)
rtcr = np_to_ufl_matrix(rtcr_np)
I = np_to_ufl_matrix(np.identity(2))

# real matrix case
# C = 10.0*I + c*cosk1 + rcrt*cosk2 + rtcr*cosk3

# complex matrix case
# sink1 = Expression('0.5*sin(k10*x[0]+k11*x[1])', k10=k1[0], k11=k1[1], degree=1)
# sink2 = Expression('0.5*sin(k20*x[0]+k21*x[1])', k20=k2[0], k21=k2[1], degree=1)
# sink3 = Expression('0.5*sin(k30*x[0]+k31*x[1])', k30=k3[0], k31=k3[1], degree=1)

# ct_np = np.transpose(c_np)
# rctrt_np = np.dot(np.dot(r_np,ct_np), r_np_t)
# rtctr_np = np.dot(np.dot(r_np_t,ct_np), r_np)

# ct = np_to_ufl_matrix(ct_np)
# rctrt = np_to_ufl_matrix(rctrt_np)
# rtctr = np_to_ufl_matrix(rtctr_np)

# C_r = 10.0*I + c*cosk1 + rcrt*cosk2 + rtcr*cosk3 + ct*cosk1 + rctrt*cosk2 + rtctr*cosk3 
# C_i = 10.0*I + c*sink1 + rcrt*sink2 + rtcr*sink3 - ct*sink1 - rctrt*sink2 - rtctr*sink3

In [14]:
## Computations

# assemble mass matrix
m_r = (u_r*v_r + u_i*v_i)*dx
m_i = (-u_r*v_i + u_i*v_r)*dx
M = PETScMatrix()
assemble(m_r+m_i, tensor=M)
    
#L = inner(Constant(1.0), u_r+u_i)*dx
#b = PETScVector()
#assemble_system(m_r+m_i, L, A_tensor=M, b_tensor=b)

# eigenvalue solver
def compute_evals(A, E, M, r):
    
    # create eigensolver
    eigensolver = SLEPcEigenSolver(A,M)
    eigensolver.parameters['spectrum'] = 'smallest real'
    eigensolver.parameters['solver']   = 'krylov-schur'    # 'krylov-schur' 'lapack'
    eigensolver.parameters['tolerance'] = 1.e-8   # 1.e-15
    #eigensolver.parameters['problem_type'] = 'hermitian'
    #eigensolver.parameters['maximum_iterations'] = 200

    # solve for eigenvalues
    eigensolver.solve(evals)
    
    for i in range(0,evals):
        if i <= (eigensolver.get_number_converged()-1):
            E[r][i] = eigensolver.get_eigenvalue(i)[0]
    
    return np.sort(E)

program_starts = time.time()

#define problem
#a_r_0 = V_e*(u_r*v_r + u_i*v_i)
#a_i_0 = V_e*(-u_r*v_i + u_i*v_r)

r = 0
# main computation
for k_vec in boundary.transpose():
    
    k = Constant(k_vec)

    # define problem
    
    a_r = ( inner(C*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) )*dx
    a_i = ( inner(C*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) - inner(C*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) )*dx

#     a_r = ( inner(C_r*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_r*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) +\
#            -inner(C_i*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) + inner(C_i*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) )*dx
#     a_i = ( inner(C_r*(grad(u_i)+k*u_r), grad(v_r)-k*v_i) - inner(C_r*(grad(u_r)-k*u_i), grad(v_i)+k*v_r) +\
#            +inner(C_i*(grad(u_r)-k*u_i), grad(v_r)-k*v_i) + inner(C_i*(grad(u_i)+k*u_r), grad(v_i)+k*v_r) )*dx
    
    # assemble stiffness matrix
    A = PETScMatrix()
    assemble(a_r+a_i, tensor=A)
    
    #assemble_system(a_r+a_i, L)#, A_tensor=A, b_tensor=b)
    
    E = compute_evals(A, E, M, r)
    
    r += 1
    print r/float(J)
    
print time.time() - program_starts

NameError: name 'C_r' is not defined

In [ ]:
## Plotting eigenvalues

plt.plot(quasimomentum, E,'k.')
#plt.plot(E[:,round(2*Ly-1)],DELTA,'r.')

#plt.ylim([50,320])

plt.show()

In [ ]:
## Plotting eigenfunctions
    
#eigensolver = SLEPcEigenSolver(A,M)
#eigensolver.parameters['spectrum'] = 'smallest real'
#eigensolver.parameters['solver']   = 'krylov-schur'
#eigensolver.parameters['tolerance'] = 1.e-10

#eigensolver.solve(evals)

##r, c, rx, cx = eigensolver.get_eigenpair(int(round(2*Ly-1)))
#r, c, rx, cx = eigensolver.get_eigenpair(2)

#u = Function(W)
#u.vector()[:] = rx
#u_r, u_i = split(u)
#plot(u_r)
##plot(mesh)
#interactive()